# Trabajo Práctico 1

Estudiantes:

- Alonso Araya Calvo
- Pedro Soto
- Sofia Oviedo

# Imports

In [ ]:
import warnings

import matplotlib.pyplot as plt
import optuna
import pandas as pd
from matplotlib.pyplot import *
from scipy import stats
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import get_file
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from optuna.visualization.matplotlib import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_contour,
    plot_slice,
    plot_edf,
)

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# Cargando dataset

In [ ]:
#tomado de https://www.kaggle.com/code/wailinnoo/intrusion-detection-system-using-kdd99-dataset
try:
    path = get_file('kddcup.data_10_percent.gz',
                    origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz')
except:
    print('Error downloading')
    raise

print(path)

# This file is a CSV, just no CSV extension or headers
# Download from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
pd_data_frame = pd.read_csv(path, header=None)

# The CSV file has no column heads, so add them
pd_data_frame.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

#describe the dataset
#transform nominal features using a one hot vector encoding
pd_data_frame.describe()

# Preprocessing and  Dummy encoding transformation

Attacks fall into four main categories:






1.   DOS: denial-of-service, e.g. syn flood;
2.   R2L: unauthorized access from a remote machine, e.g. guessing password;

3. U2R:  unauthorized access to local superuser (root) privileges, e.g., various ``buffer overflow'' attacks;

4. probing: surveillance and other probing, e.g., port scanning.


It is important to note that the test data is not from the same probability distribution as the training data, and it includes specific attack types not in the training data.  This makes the task more realistic.  Some intrusion experts believe that most novel attacks are variants of known attacks and the "signature" of known attacks can be sufficient to catch novel variants.  The datasets contain a total of 24 training attack types, with an additional 14 types in the test data only.

Neptune is the most frequent attack in the dataset, consisting in a type of denial-of-service (DoS) attack overwhelming systems with SYN requests (ynchronize request, is a type of network packet used in the TCP handshake to initiate a connection between a client and a server).

**In this work we will focus in the detection of backdrop attacks**.

In [ ]:
# For now, just drop NA's (rows with missing values), in case there are
pd_data_frame.dropna(inplace=True, axis=1)

# Checkng for DUPLICATE values
pd_data_frame.drop_duplicates(keep='first', inplace=True)
print(pd_data_frame.describe())
filtered_df = pd_data_frame

In [ ]:
# distribution of the attack categories (outcome)
# Exploratory data analysis
plt.figure(figsize=(15, 7))
class_distribution = pd_data_frame['outcome'].value_counts()
class_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Data points per Class')
plt.title('Distribution of yi in train data')
plt.grid()
plt.show()

#normal and neptune are the more prominent classes

In [ ]:
list_nominal_features = ["flag", "protocol_type", "service"]

# Apply one-hot encoding to the nominal features
df_encoded = pd.get_dummies(filtered_df, columns=list_nominal_features)

# Convert boolean columns (from one-hot encoding) to integers (0 or 1) in df_encoded
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'bool':
        df_encoded[col] = df_encoded[col].astype(int)

# Display the first few rows of the modified DataFrame to verify
print("DataFrame with boolean columns converted to integers:")
display(df_encoded.describe())
print("Columns after nominal attributes encoded: ")
for i in df_encoded.columns:
    print(i)


# 1. (30 puntos) Implementacion de un arbol de decision y random forests para clasificar todos los tipos de ataques (scikit learn)

## 1.1 Genere una funcion split_dataset la cual divida los datos en entrenamiento (70 %), validacion (15 %) y prueba (15 %).

In [ ]:
def split_dataset(df, target_column='outcome', test_size=0.15, val_size=0.15, random_state=42, show_sizes=False):
    """
    Función que parte el dataset en conjuntos de entrenamiento, validación y prueba
    por medio de los parametros solicitados.
    
    Parámetros:
        df : DataFrame de pandas con los datos del dataset
        target_column : Nombre de la columna de las clases en string
        test_size : Proporción numerica del conjunto de prueba
        val_size : Proporción numerica del conjunto de validación
        random_state : Seed para poder reproducir los resultados
    
    Salida:
        tupla con esta forma:
            (X_train, X_val, X_test, y_train, y_val, y_test)

        X_train: Dataframe con las caracteristicas de entrenamiento
        X_val: Dataframe con las caracteristicas de validación
        X_test: Dataframe con las caracteristicas de prueba
        y_train: Series con las clases de entrenamiento
        y_val: Series con las clases de validación
        y_test: Series con las clases de prueba
    """

    # Separo las caracteristicas de KDD99 y las clases de outcome
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # En el primer split se separan los datos
    # de entrenamiento y validación de los de prueba
    # generando el split de prueba
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )

    # Ajustar el tamaño de validación respecto al conjunto temporal
    val_size_adjusted = val_size / (1 - test_size)  # 0.15 / 0.85 ≈ 0.176

    # Para este segundo split se separa el conjunto de entrenamiento
    # y validación, generando el split de validación y entrenamiento
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=val_size_adjusted,
        random_state=random_state,
        stratify=y_temp
    )
    if show_sizes:
        print(f"Tamaño del conjunto completo: {len(df)}")
        print(f"Entrenamiento: {len(X_train)} ({len(X_train) / len(df) * 100}%)")
        print(f"Validación: {len(X_val)} ({len(X_val) / len(df) * 100}%)")
        print(f"Prueba: {len(X_test)} ({len(X_test) / len(df) * 100}%)")

    return X_train, X_val, X_test, y_train, y_val, y_test


X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(df_encoded, show_sizes=True)

# Imprime la distribución de clases para las particiones
print("\n" + "=" * 50)
print("Distribucion de clases en los distintos dataset")
print("=" * 50)

print("\nDistribución en conjunto completo:")
print(df_encoded['outcome'].value_counts(normalize=True).sort_index())

print("\nDistribución en conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True).sort_index())

print("\nDistribución en conjunto de validación:")
print(y_val.value_counts(normalize=True).sort_index())

print("\nDistribución en conjunto de prueba:")
print(y_test.value_counts(normalize=True).sort_index())

## 1.2 (15 puntos) Entrene un arbol de decision (de scikitlearn) para clasificar los ataques en todas las categorias originales del dataset:

a) Optimice la profundidad maxima, cantidad minima de observaciones
por particion, y el criterio de pureza usando optuna o weights and
biases. Documente los rangos de cada hiper-parametro y justifique la
decision por cada uno. Muestre los graficos de ese proceso de optimizacion
y seleccione las 3 mejores arquitecturas.

b) Compare las tres mejores arquitecturas, para al menos 10 corridas
diferentes (particiones), el F1-score promedio para todas las clases
y la tasa de falsos positivos promedio para todas las clases, presente
medias y desviaciones estandar usando la particion de prueba.
Comente los resultados.

### Optimización con Optuna 

In [ ]:
def optimize_decision_tree(trial):
    """
    Función para optimizar hiperparámetros del árbol de decisión usando Optuna.
    
    Parámetros:
        trial: Objeto de Optuna que sugiere valores para los hiperparámetros
        
    Retorna:
        f1_macro: F1-Score promedio macro en el conjunto de validación
    """
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 25)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Crear el modelo con hiperparámetros sugeridos
    dt = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )

    # Entrenar el modelo
    dt.fit(X_train, y_train)

    # Evaluar en conjunto de validación
    y_pred = dt.predict(X_val)

    # Se calcula el F1-Score promedio macro para todas las clases
    f1_macro = f1_score(y_val, y_pred, average='macro')

    return f1_macro

In [ ]:
# Se ejecuta optuna para crear las optimizaciones
study = optuna.create_study(
    direction='maximize',  # Maximizar F1-score
    study_name='DecisionTree_KDD99_Optimization',
    storage='sqlite:///decision_tree_optimization.db'
)

print("Iniciando optimización de hiperparámetros con Optuna...")

# 100 pruebas de Optuna que son suficientes para encontrar las mejores arquitecturas
n_trials = 100
study.optimize(optimize_decision_tree, n_trials=n_trials, show_progress_bar=True)

print("\n" + "=" * 60)
print("Resultados de la optimización con Optuna")
print("=" * 60)
print(f"Número total de trials: {len(study.trials)}")
print(f"Mejor F1-macro encontrado: {study.best_value:.4f}")
print("\nMejores hiperparámetros:")
for param, value in study.best_params.items():
    print(f"  {param}: {value}")

# Obtener las 3 mejores arquitecturas
best_trials = sorted(study.trials, key=lambda x: x.value if x.value is not None else -1, reverse=True)[:3]

print("\n" + "=" * 60)
print("Las 3 mejores arquitecturas encontradas")
print("=" * 60)

top_3_configs = []
for i, trial in enumerate(best_trials, 1):
    print(f"\nArquitectura #{i}:")
    print(f"  F1-macro: {trial.value:.4f}")
    print("  Hiperparámetros:")
    for param, value in trial.params.items():
        print(f"    {param}: {value}")

    # Guardar configuración para usar los datos en las celdas siguientes
    top_3_configs.append({
        'params': trial.params,
        'f1_score': trial.value,
        'rank': i
    })

### Estadisticas y Graficos de Proceso de Optimizacion

In [ ]:
# Visualizaciones del proceso de optimización con Optuna

# 1) Historia de la optimización
plot_optimization_history(study); plt.show()

# 2) Importancia de hiperparámetros
plot_param_importances(study); plt.show()

# 3) Relaciones entre hiperparámetros
plot_parallel_coordinate(study); plt.show()
plot_contour(
    study,
    params=['max_depth', 'min_samples_split', 'min_samples_leaf', 'criterion']
); plt.show()
plot_slice(study); plt.show()

# 4) Distribución de valores objetivo 
plot_edf(study); plt.show()

# 5) Comparación visual de las 3 mejores arquitecturas (se mantiene)
top_3_f1 = [config['f1_score'] for config in top_3_configs]
top_3_names = [f"Arquitectura #{config['rank']}" for config in top_3_configs]

plt.figure(figsize=(6, 4))
bars = plt.bar(top_3_names, top_3_f1, color=['gold', 'silver', '#CD7F32'], alpha=0.8)
plt.ylabel('F1-score')
plt.title('Las 3 Mejores Arquitecturas')
plt.xticks(rotation=45)

for bar, value in zip(bars, top_3_f1):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.0005,
             f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Estadísticas de la optimización
f1_scores = [trial.value for trial in study.trials if trial.value is not None]
print("\n" + "=" * 60)
print("Estadísticas de la optimización")
print("=" * 60)
print(f"F1-score promedio: {np.mean(f1_scores):.4f}")
print(f"Desviación estándar: {np.std(f1_scores):.4f}")
print(f"F1-score mínimo: {np.min(f1_scores):.4f}")
print(f"F1-score máximo: {np.max(f1_scores):.4f}")
print(f"Mejora sobre promedio: {((study.best_value - np.mean(f1_scores)) / np.mean(f1_scores) * 100):.2f}%")

### Comparacion y Evaluacion de las Arquitecturas con Particiones Diferentes

In [ ]:
def evaluate_dt_architecture_multiple_runs(params, n_runs=10, random_seeds=None):
    """
    Evalúa una arquitectura de árbol de decisión con múltiples corridas aleatorias.
    
    Parámetros:
        params: Diccionario con hiperparámetros del árbol
        n_runs: Número de corridas independientes (default: 10)
        random_seeds: Lista de seeds aleatorios (opcional)
        
    Retorna:
        results: Diccionario con todas las metricas recopiladas y parametros utilizados
    """

    if random_seeds is None:
        random_seeds = list(range(42, 42 + n_runs))

    all_f1_scores = []
    all_accuracies = []
    all_fpr_rates = []  # Tasas de Falsos Positivos
    all_detailed_metrics = []

    print(f"Evaluando arquitectura con {n_runs} corridas independientes...")

    for run in range(n_runs):
        seed = random_seeds[run]

        # Crear nueva partición aleatoria de datos
        X_train_run, X_val_run, X_test_run, y_train_run, y_val_run, y_test_run = split_dataset(
            df_encoded, random_state=seed
        )

        # Crear y entrenar modelo con parámetros dados
        dt = DecisionTreeClassifier(**params, random_state=seed)
        dt.fit(X_train_run, y_train_run)

        # Predicciones en conjunto de prueba
        y_pred_test = dt.predict(X_test_run)

        # Calcular métricas
        f1_macro = f1_score(y_test_run, y_pred_test, average='macro')
        accuracy = accuracy_score(y_test_run, y_pred_test)

        # Métricas por clase
        f1_per_class = f1_score(y_test_run, y_pred_test, average=None, labels=dt.classes_)
        precision_per_class, recall_per_class, _, _ = precision_recall_fscore_support(
            y_test_run, y_pred_test, average=None, labels=dt.classes_
        )

        # Calcular matriz de confusión
        cm = confusion_matrix(y_test_run, y_pred_test, labels=dt.classes_)

        # Calcular tasa de falsos positivos por clase
        fpr_per_class = []
        for i in range(len(dt.classes_)):
            # FPR = FP / (FP + TN)
            fp = cm[:, i].sum() - cm[i, i]  # Falsos positivos para clase i
            tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])  # Verdaderos negativos
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
            fpr_per_class.append(fpr)

        avg_fpr = np.mean(fpr_per_class)

        # Recopilar los resultados de la corrida
        all_f1_scores.append(f1_macro)
        all_accuracies.append(accuracy)
        all_fpr_rates.append(avg_fpr)
        all_detailed_metrics.append({
            'run': run + 1,
            'seed': seed,
            'f1_macro': f1_macro,
            'accuracy': accuracy,
            'avg_fpr': avg_fpr,
            'f1_per_class': f1_per_class,
            'precision_per_class': precision_per_class,
            'recall_per_class': recall_per_class,
            'fpr_per_class': fpr_per_class,
            'classes': dt.classes_
        })

        print(f"Corrida {run + 1:2d}/{n_runs} - F1: {f1_macro:.4f}, Accuracy: {accuracy:.4f}, FPR: {avg_fpr:.4f}")

    # Calcular estadísticas agregadas
    results = {
        'n_runs': n_runs,
        'params': params,
        'f1_scores': all_f1_scores,
        'accuracies': all_accuracies,
        'fpr_rates': all_fpr_rates,
        'detailed_metrics': all_detailed_metrics,
        'statistics': {
            'f1_mean': np.mean(all_f1_scores),
            'f1_std': np.std(all_f1_scores),
            'accuracy_mean': np.mean(all_accuracies),
            'accuracy_std': np.std(all_accuracies),
            'fpr_mean': np.mean(all_fpr_rates),
            'fpr_std': np.std(all_fpr_rates)
        }
    }

    return results

In [ ]:
# Evaluar las 3 mejores arquitecturas con 10 corridas cada una
print("=" * 70)
print("Evaluación de las 3 mejores arquitecturas con 10 corridas")
print("=" * 70)

evaluation_results = []

for i, config in enumerate(top_3_configs):
    print(f"\n{'-' * 50}")
    print(f"Evaluando Arquitectura #{config['rank']}")
    print("Hiperparámetros:")
    for param, value in config['params'].items():
        print(f"  {param}: {value}")
    print(f"{'-' * 50}")

    # Evaluar con 10 corridas independientes usando seeds únicos
    results = evaluate_dt_architecture_multiple_runs(
        params=config['params'],
        n_runs=10,
        random_seeds=list(range(100 + i * 10, 110 + i * 10))
    )

    # Añadir información de las corridas originales
    results['architecture_rank'] = config['rank']
    results['optimization_f1'] = config['f1_score']
    evaluation_results.append(results)

### Resumen de Resultados

In [ ]:
print("\n" + "=" * 90)
print("Tabla resumen de resultados - Evaluación de árboles de decisión")
print("=" * 90)

# Generar tabla con estadísticas previamente recopiladas
summary_data = []
for result in evaluation_results:
    summary_data.append({
        'Arquitectura': f"#{result['architecture_rank']}",
        'F1-macro Media': f"{result['statistics']['f1_mean']:.4f}",
        'F1-macro Desviación Estandar': f"{result['statistics']['f1_std']:.4f}",
        'Accuracy Media': f"{result['statistics']['accuracy_mean']:.4f}",
        'Accuracy Desviación Estandar': f"{result['statistics']['accuracy_std']:.4f}",
        'FPR Media': f"{result['statistics']['fpr_mean']:.4f}",
        'FPR Desviación Estandar': f"{result['statistics']['fpr_std']:.4f}",
        'max_depth': result['params']['max_depth'],
        'min_samples_split': result['params']['min_samples_split'],
        'min_samples_leaf': result['params']['min_samples_leaf'],
        'criterion': result['params']['criterion']
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación de F1-scores con barras de error
architectures = [f"Arq #{r['architecture_rank']}" for r in evaluation_results]
f1_means = [r['statistics']['f1_mean'] for r in evaluation_results]
f1_stds = [r['statistics']['f1_std'] for r in evaluation_results]

bars1 = axes[0, 0].bar(architectures, f1_means, yerr=f1_stds, capsize=8,
                       color=['gold', 'silver', '#CD7F32'], alpha=0.8, edgecolor='black')
axes[0, 0].set_ylabel('F1-score')
axes[0, 0].set_title('Comparación F1-score (Media ± Desviación Estándar)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

for bar, mean, std in zip(bars1, f1_means, f1_stds):
    axes[0, 0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + std + 0.001,
                    f'{mean:.4f}', ha='center', va='bottom', fontweight='bold')

# 2. Comparación de tasas de falsos positivos
fpr_means = [r['statistics']['fpr_mean'] for r in evaluation_results]
fpr_stds = [r['statistics']['fpr_std'] for r in evaluation_results]

bars2 = axes[0, 1].bar(architectures, fpr_means, yerr=fpr_stds, capsize=8,
                       color=['gold', 'silver', '#CD7F32'], alpha=0.8, edgecolor='black')
axes[0, 1].set_ylabel('Tasa de Falsos Positivos')
axes[0, 1].set_title('Comparación FPR (Media ± Desviación Estándar)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

for bar, mean, std in zip(bars2, fpr_means, fpr_stds):
    axes[0, 1].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + std + 0.0005,
                    f'{mean:.4f}', ha='center', va='bottom', fontweight='bold')

# 3. Distribuciones de F1-scores (Box plots)
f1_data = [r['f1_scores'] for r in evaluation_results]
box1 = axes[1, 0].boxplot(f1_data, labels=architectures, patch_artist=True)
colors = ['gold', 'silver', '#CD7F32']
for patch, color in zip(box1['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)
axes[1, 0].set_ylabel('F1-score')
axes[1, 0].set_title('Distribución de F1-scores por Arquitectura')
axes[1, 0].grid(True, alpha=0.3)

# 4. Distribuciones de FPR (Box plots)
fpr_data = [r['fpr_rates'] for r in evaluation_results]
box2 = axes[1, 1].boxplot(fpr_data, labels=architectures, patch_artist=True)
for patch, color in zip(box2['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)
axes[1, 1].set_ylabel('Tasa de Falsos Positivos')
axes[1, 1].set_title('Distribución de FPR por Arquitectura')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identificar mejor arquitectura
best_arch_idx = np.argmax([r['statistics']['f1_mean'] for r in evaluation_results])
best_arch = evaluation_results[best_arch_idx]

print(f"\n{'=' * 80}")
print("Arquitectura recomendada")
print("=" * 80)
print(f"Mejor arquitectura: #{best_arch['architecture_rank']}")
print(f"F1-score: {best_arch['statistics']['f1_mean']:.4f} ± {best_arch['statistics']['f1_std']:.4f}")
print(f"FPR: {best_arch['statistics']['fpr_mean']:.4f} ± {best_arch['statistics']['fpr_std']:.4f}")
print(f"Accuracy: {best_arch['statistics']['accuracy_mean']:.4f} ± {best_arch['statistics']['accuracy_std']:.4f}")
print("\nHiperparámetros óptimos:")
for param, value in best_arch['params'].items():
    print(f"  {param}: {value}")

## 1.3. (15 puntos) Usando tambien la libreria scikit learn entrene un random forest.

a) Optimice con optuna o weights and biases la cantidad de arboles del
random forest. Defina el rango de numero de arboles de decision de
forma justificada.

b) Compare los 2 mejores random forests seleccionados por la herramienta,
haciendo 10 corridas diferentes (particiones), el F1-score promedio
para todas las clases y la tasa de falsos positivos promedio
para todas las clases, presente medias y desviaciones estandar usando
la particion de prueba. Comente los resultados.

### Optimización con Optuna

In [ ]:
def objective_random_forest(trial):
    """
    Función para optimizar solo los n_estimators del Random Forest usando Optuna.
    
    Parámetros:
        trial: Objeto de Optuna que sugiere valores para n_estimators
        
    Retorna:
        f1_macro: F1-Score promedio macro en el conjunto de validación
    """
    
    # Rango de numero de arboles
    n_estimators = trial.suggest_int('n_estimators', 10, 300)
    
    # Creacion de RandomForest
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        random_state=42,
        n_jobs=-1
    )
    
    # Entrenar el modelo
    rf.fit(X_train, y_train)
    
    # Evaluar en conjunto de validación
    y_pred = rf.predict(X_val)
    
    # Calcular F1-Score promedio macro para todas las clases
    f1_macro = f1_score(y_val, y_pred, average='macro')
    
    return f1_macro

In [ ]:
# Ejecutar optimización de Random Forest
study_rf = optuna.create_study(
    direction='maximize',  # Maximizar F1-score
    study_name='RandomForest_KDD99_Optimization',
    storage='sqlite:///random_forest_optimization.db'
)

print("Iniciando optimización de hiperparámetros de Random Forest con Optuna...")
print("=" * 80)

# 50 pruebas de optimización debido al alto tiempo de ejecucion y computo
# utilizado en comparacion a entrenar un arbol de decision.
n_trials_rf = 50
study_rf.optimize(objective_random_forest, n_trials=n_trials_rf, show_progress_bar=True)

print("\n" + "=" * 60)
print("Resultados de la optimización de Random Forest con Optuna")
print("=" * 60)
print(f"Número total de trials: {len(study_rf.trials)}")
print(f"Mejor F1-macro encontrado: {study_rf.best_value:.4f}")
print("\nMejores hiperparámetros:")
for param, value in study_rf.best_params.items():
    print(f"  {param}: {value}")

# Obtener los 2 mejores Random Forests
best_trials_rf = sorted(study_rf.trials, key=lambda x: x.value if x.value is not None else -1, reverse=True)[:2]

print("\n" + "=" * 60)
print("Los 2 mejores Random Forests encontrados")
print("=" * 60)

top_2_rf_configs = []
for i, trial in enumerate(best_trials_rf, 1):
    print(f"\nRandom Forest #{i}:")
    print(f"  F1-macro: {trial.value:.4f}")
    print("  Hiperparámetros:")
    for param, value in trial.params.items():
        print(f"    {param}: {value}")

    # Guardar configuración
    top_2_rf_configs.append({
        'params': trial.params,
        'f1_score': trial.value,
        'rank': i
    })

### Visualizacion del Proceso de Optimizacion

In [ ]:
# Visualizaciones del proceso de optimización con Optuna

# 1) Historia de la optimización de Random Forest
plot_optimization_history(study_rf); plt.show()

# 2) Importancia de hiperparámetros de Random Forest
plot_param_importances(study_rf); plt.show()

# 3) Análisis slice de cada hiperparámetro
plot_slice(study_rf); plt.show()

# 4) Distribución empírica de valores objetivo (EDF)
plot_edf(study_rf); plt.show()

# 5) Comparación visual de los 2 mejores Random Forests
top_2_f1_rf = [config['f1_score'] for config in top_2_rf_configs]
top_2_names_rf = [f"RF #{config['rank']}" for config in top_2_rf_configs]

plt.figure(figsize=(8, 6))
bars = plt.bar(top_2_names_rf, top_2_f1_rf, color=['gold', 'silver'], alpha=0.8, edgecolor='black')
plt.ylabel('F1-score')
plt.title('Los 2 Mejores Random Forests')
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores sobre las barras
for bar, value in zip(bars, top_2_f1_rf):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.0005,
             f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Estadísticas de la optimización RF
f1_scores_rf = [trial.value for trial in study_rf.trials if trial.value is not None]
print("\n" + "=" * 60)
print("Estadísticas de la optimización de Random Forest")
print("=" * 60)
print(f"F1-score promedio: {np.mean(f1_scores_rf):.4f}")
print(f"Desviación estándar: {np.std(f1_scores_rf):.4f}")
print(f"F1-score mínimo: {np.min(f1_scores_rf):.4f}")
print(f"F1-score máximo: {np.max(f1_scores_rf):.4f}")
print(f"Mejora sobre promedio: {((study_rf.best_value - np.mean(f1_scores_rf)) / np.mean(f1_scores_rf) * 100):.2f}%")

# Análisis específico de n_estimators encontrados
n_estimators_values = [trial.params.get('n_estimators', 0) for trial in study_rf.trials]
print(f"\n" + "=" * 60)
print("Analisis de numero de arboles")
print("=" * 60)
print(f"Rango explorado: {min(n_estimators_values)} - {max(n_estimators_values)}")
print(f"Promedio de trials: {np.mean(n_estimators_values):.1f}")
print(f"Mediana de trials: {np.median(n_estimators_values):.1f}")
print(f"Desviación estándar: {np.std(n_estimators_values):.1f}")
print(f"Mejor configuración: {study_rf.best_params['n_estimators']} árboles")

### Evaluacion de Mejores Arquitecturas con Particiones Independientes

In [ ]:
def evaluate_rf_architecture_multiple_runs(params, n_runs=10, random_seeds=None):
    """
    Evalúa una arquitectura de Random Forest con múltiples corridas aleatorias.
    
    Parámetros:
        params: Diccionario con hiperparámetros del Random Forest
        n_runs: Número de corridas independientes
        random_seeds: Lista de seeds aleatorios
        
    Retorna:
        results: Diccionario con métricas recopiladas y parametros utilizados
    """

    if random_seeds is None:
        random_seeds = list(range(200, 200 + n_runs))

    all_f1_scores = []
    all_accuracies = []
    all_fpr_rates = []
    all_detailed_metrics = []

    print(f"Evaluando Random Forest con {n_runs} corridas independientes...")

    for run in range(n_runs):
        seed = random_seeds[run]

        # Crear nueva partición aleatoria de datos
        X_train_run, X_val_run, X_test_run, y_train_run, y_val_run, y_test_run = split_dataset(
            df_encoded, random_state=seed
        )

        # Crear y entrenar Random Forest con parámetros dados
        rf = RandomForestClassifier(**params, random_state=seed, n_jobs=-1)
        rf.fit(X_train_run, y_train_run)

        # Predicciones en conjunto de prueba
        y_pred_test = rf.predict(X_test_run)

        # Calcular métricas
        f1_macro = f1_score(y_test_run, y_pred_test, average='macro')
        accuracy = accuracy_score(y_test_run, y_pred_test)

        # Métricas por clase
        f1_per_class = f1_score(y_test_run, y_pred_test, average=None, labels=rf.classes_)
        precision_per_class, recall_per_class, _, _ = precision_recall_fscore_support(
            y_test_run, y_pred_test, average=None, labels=rf.classes_
        )

        # Calcular matriz de confusión
        cm = confusion_matrix(y_test_run, y_pred_test, labels=rf.classes_)

        # Calcular tasa de falsos positivos por clase
        fpr_per_class = []
        for i in range(len(rf.classes_)):
            fp = cm[:, i].sum() - cm[i, i]  # Falsos positivos para clase i
            tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])  # Verdaderos negativos
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
            fpr_per_class.append(fpr)

        avg_fpr = np.mean(fpr_per_class)

        # Recopilar resultados de la corrida
        all_f1_scores.append(f1_macro)
        all_accuracies.append(accuracy)
        all_fpr_rates.append(avg_fpr)
        all_detailed_metrics.append({
            'run': run + 1,
            'seed': seed,
            'f1_macro': f1_macro,
            'accuracy': accuracy,
            'avg_fpr': avg_fpr,
            'f1_per_class': f1_per_class,
            'precision_per_class': precision_per_class,
            'recall_per_class': recall_per_class,
            'fpr_per_class': fpr_per_class,
            'classes': rf.classes_
        })

        print(f"Corrida {run + 1:2d}/{n_runs} - F1: {f1_macro:.4f}, Accuracy: {accuracy:.4f}, FPR: {avg_fpr:.4f}")

    # Calcular estadísticas agregadas
    results = {
        'n_runs': n_runs,
        'params': params,
        'f1_scores': all_f1_scores,
        'accuracies': all_accuracies,
        'fpr_rates': all_fpr_rates,
        'detailed_metrics': all_detailed_metrics,
        'statistics': {
            'f1_mean': np.mean(all_f1_scores),
            'f1_std': np.std(all_f1_scores),
            'accuracy_mean': np.mean(all_accuracies),
            'accuracy_std': np.std(all_accuracies),
            'fpr_mean': np.mean(all_fpr_rates),
            'fpr_std': np.std(all_fpr_rates)
        }
    }

    return results

In [ ]:
# Evaluar los 2 mejores Random Forests con 10 corridas cada uno
print("=" * 70)
print("Evaluación de los 2 mejores Random Forests con 10 corridas")
print("=" * 70)

rf_evaluation_results = []

for i, config in enumerate(top_2_rf_configs):
    print(f"\n{'-' * 50}")
    print(f"Evaluando Random Forest #{config['rank']}")
    print("Hiperparámetros:")
    for param, value in config['params'].items():
        print(f"  {param}: {value}")
    print(f"{'-' * 50}")

    # Evaluar con 10 corridas independientes usando seeds únicos
    results = evaluate_rf_architecture_multiple_runs(
        params=config['params'],
        n_runs=10,
        random_seeds=list(range(300 + i * 10, 310 + i * 10))  # Seeds únicos para cada RF
    )

    # Añadir información de corridas originales
    results['architecture_rank'] = config['rank']
    results['optimization_f1'] = config['f1_score']
    rf_evaluation_results.append(results)

### Resumen de Resultados

In [ ]:
# Crear tabla resumen para Random Forests
print("\n" + "=" * 90)
print("Resumen Evaluacion Random Forests")
print("=" * 90)

# Generar tabla con estadísticas detalladas para RF
rf_summary_data = []
for result in rf_evaluation_results:
    rf_summary_data.append({
        'Random Forest': f"#{result['architecture_rank']}",
        'n_estimators': result['params']['n_estimators'],
        'F1-macro Media': f"{result['statistics']['f1_mean']:.4f}",
        'F1-macro Desv Std': f"{result['statistics']['f1_std']:.4f}",
        'Accuracy Media': f"{result['statistics']['accuracy_mean']:.4f}",
        'Accuracy Desv Std': f"{result['statistics']['accuracy_std']:.4f}",
        'FPR Media': f"{result['statistics']['fpr_mean']:.4f}",
        'FPR Desv Std': f"{result['statistics']['fpr_std']:.4f}",
        'F1 Optimización': f"{result['optimization_f1']:.4f}"
    })

rf_summary_df = pd.DataFrame(rf_summary_data)
print(rf_summary_df.to_string(index=False))

# Visualizaciones comparativas detalladas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación F1-scores RF con barras de error
rf_architectures = [f"RF #{r['architecture_rank']}\n(n={r['params']['n_estimators']})" 
                   for r in rf_evaluation_results]
rf_f1_means = [r['statistics']['f1_mean'] for r in rf_evaluation_results]
rf_f1_stds = [r['statistics']['f1_std'] for r in rf_evaluation_results]

bars1 = axes[0, 0].bar(rf_architectures, rf_f1_means, yerr=rf_f1_stds, capsize=8,
                       color=['gold', 'silver'], alpha=0.8, edgecolor='black')
axes[0, 0].set_ylabel('F1-score')
axes[0, 0].set_title('Comparación F1-score Random Forests\n(Media ± Desviación Estándar)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Añadir valores sobre las barras
for bar, mean, std in zip(bars1, rf_f1_means, rf_f1_stds):
    axes[0, 0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + std + 0.001,
                    f'{mean:.4f}', ha='center', va='bottom', fontweight='bold')

# 2. Comparación FPR RF
rf_fpr_means = [r['statistics']['fpr_mean'] for r in rf_evaluation_results]
rf_fpr_stds = [r['statistics']['fpr_std'] for r in rf_evaluation_results]

bars2 = axes[0, 1].bar(rf_architectures, rf_fpr_means, yerr=rf_fpr_stds, capsize=8,
                       color=['gold', 'silver'], alpha=0.8, edgecolor='black')
axes[0, 1].set_ylabel('Tasa de Falsos Positivos')
axes[0, 1].set_title('Comparación FPR Random Forests\n(Media ± Desviación Estándar)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

for bar, mean, std in zip(bars2, rf_fpr_means, rf_fpr_stds):
    axes[0, 1].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + std + 0.0005,
                    f'{mean:.4f}', ha='center', va='bottom', fontweight='bold')

# 3. Box plots F1-scores RF
rf_f1_data = [r['f1_scores'] for r in rf_evaluation_results]
box1 = axes[1, 0].boxplot(rf_f1_data, labels=[f"RF #{r['architecture_rank']}" 
                                             for r in rf_evaluation_results], patch_artist=True)
colors_rf = ['gold', 'silver']
for patch, color in zip(box1['boxes'], colors_rf):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)
axes[1, 0].set_ylabel('F1-score')
axes[1, 0].set_title('Distribución F1-scores Random Forests')
axes[1, 0].grid(True, alpha=0.3)

# 4. Box plots FPR RF
rf_fpr_data = [r['fpr_rates'] for r in rf_evaluation_results]
box2 = axes[1, 1].boxplot(rf_fpr_data, labels=[f"RF #{r['architecture_rank']}" 
                                              for r in rf_evaluation_results], patch_artist=True)
for patch, color in zip(box2['boxes'], colors_rf):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)
axes[1, 1].set_ylabel('Tasa de Falsos Positivos')
axes[1, 1].set_title('Distribución FPR Random Forests')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## Análisis Estadístico y Comentarios de Resultados

# Análisis estadístico de diferencias entre RF
print("\n" + "=" * 80)
print("ANÁLISIS ESTADÍSTICO DE DIFERENCIAS ENTRE RANDOM FORESTS")
print("=" * 80)

rf_f1_1 = rf_evaluation_results[0]['f1_scores']
rf_f1_2 = rf_evaluation_results[1]['f1_scores']

try:
    # t-test pareado para comparar los dos RF
    t_stat_rf, p_val_rf = stats.ttest_rel(rf_f1_1, rf_f1_2)
    print(f"Prueba t pareada entre RF #1 y RF #2:")
    print(f"  t-statistic: {t_stat_rf:.4f}")
    print(f"  p-value: {p_val_rf:.6f}")
    print(f"  Diferencias significativas: {'Sí' if p_val_rf < 0.05 else 'No'} (α=0.05)")
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((np.var(rf_f1_1) + np.var(rf_f1_2)) / 2)
    cohens_d = (np.mean(rf_f1_1) - np.mean(rf_f1_2)) / pooled_std
    print(f"  Effect size (Cohen's d): {cohens_d:.4f}")
    
    effect_interpretation = "pequeño" if abs(cohens_d) < 0.2 else "medio" if abs(cohens_d) < 0.8 else "grande"
    print(f"  Interpretación del efecto: {effect_interpretation}")

except Exception as e:
    print(f"Error en análisis estadístico: {e}")

# Análisis de estabilidad RF
print(f"\n{'=' * 80}")
print("ANÁLISIS DE ESTABILIDAD DE RANDOM FORESTS")
print("=" * 80)

for i, result in enumerate(rf_evaluation_results):
    rf_num = result['architecture_rank']
    n_trees = result['params']['n_estimators']
    f1_cv = result['statistics']['f1_std'] / result['statistics']['f1_mean']
    fpr_cv = result['statistics']['fpr_std'] / result['statistics']['fpr_mean'] if result['statistics']['fpr_mean'] > 0 else 0

    print(f"\nRandom Forest #{rf_num} (n_estimators={n_trees}):")
    print(f"  Coeficiente de variación F1: {f1_cv:.4f} ({'Muy estable' if f1_cv < 0.005 else 'Estable' if f1_cv < 0.01 else 'Moderada' if f1_cv < 0.02 else 'Inestable'})")
    print(f"  Coeficiente de variación FPR: {fpr_cv:.4f}")

    # Intervalos de confianza (95%)
    f1_ci_lower = result['statistics']['f1_mean'] - 1.96 * result['statistics']['f1_std']
    f1_ci_upper = result['statistics']['f1_mean'] + 1.96 * result['statistics']['f1_std']
    print(f"  Intervalo confianza F1 (95%): [{f1_ci_lower:.4f}, {f1_ci_upper:.4f}]")
    
    fpr_ci_lower = result['statistics']['fpr_mean'] - 1.96 * result['statistics']['fpr_std']
    fpr_ci_upper = result['statistics']['fpr_mean'] + 1.96 * result['statistics']['fpr_std']
    print(f"  Intervalo confianza FPR (95%): [{max(0, fpr_ci_lower):.4f}, {fpr_ci_upper:.4f}]")

# Identificar mejor Random Forest
best_rf_idx = np.argmax([r['statistics']['f1_mean'] for r in rf_evaluation_results])
best_rf = rf_evaluation_results[best_rf_idx]

print(f"\n{'=' * 80}")
print("RANDOM FOREST RECOMENDADO")
print("=" * 80)
print(f"Mejor Random Forest: #{best_rf['architecture_rank']}")
print(f"Configuración óptima: n_estimators = {best_rf['params']['n_estimators']}")
print(f"F1-score: {best_rf['statistics']['f1_mean']:.4f} ± {best_rf['statistics']['f1_std']:.4f}")
print(f"FPR: {best_rf['statistics']['fpr_mean']:.4f} ± {best_rf['statistics']['fpr_std']:.4f}")
print(f"Accuracy: {best_rf['statistics']['accuracy_mean']:.4f} ± {best_rf['statistics']['accuracy_std']:.4f}")

# Análisis de eficiencia computacional
print(f"\n{'=' * 80}")
print("ANÁLISIS DE EFICIENCIA COMPUTACIONAL")
print("=" * 80)
for result in rf_evaluation_results:
    n_trees = result['params']['n_estimators']
    f1_mean = result['statistics']['f1_mean']
    
    # Estimación relativa de complejidad (linear con n_estimators)
    complexity_ratio = n_trees / min([r['params']['n_estimators'] for r in rf_evaluation_results])
    efficiency_score = f1_mean / complexity_ratio  # F1 per unit complexity
    
    print(f"RF #{result['architecture_rank']} (n={n_trees}):")
    print(f"  Complejidad relativa: {complexity_ratio:.2f}x")
    print(f"  Eficiencia (F1/complejidad): {efficiency_score:.4f}")

print(f"\n{'=' * 80}")
print("COMENTARIOS SOBRE LOS RESULTADOS DE RANDOM FOREST")
print("=" * 80)

# Generar comentarios dinámicos basados en resultados
best_n = best_rf['params']['n_estimators']
best_f1 = best_rf['statistics']['f1_mean']
best_fpr = best_rf['statistics']['fpr_mean']

print(f"""
ANÁLISIS DE RENDIMIENTO DE RANDOM FOREST:

1. EFECTIVIDAD DEL NÚMERO DE ÁRBOLES:
   • Configuración óptima: {best_n} árboles
   • F1-score logrado: {best_f1:.4f} ({'excelente' if best_f1 > 0.995 else 'muy bueno' if best_f1 > 0.99 else 'bueno'})
   • Tasa de falsos positivos: {best_fpr:.4f} ({'muy baja' if best_fpr < 0.005 else 'baja' if best_fpr < 0.01 else 'moderada'})

2. ANÁLISIS DEL NÚMERO ÓPTIMO DE ÁRBOLES:
   • {'Configuración conservadora' if best_n < 100 else 'Configuración moderada' if best_n < 200 else 'Configuración intensiva'}
   • {'Eficiente computacionalmente' if best_n < 150 else 'Balance rendimiento-eficiencia' if best_n < 250 else 'Enfoque en máximo rendimiento'}
   • {'Apropiado para sistemas en tiempo real' if best_n < 100 else 'Adecuado para sistemas offline' if best_n > 200 else 'Versátil para ambos casos'}

3. ESTABILIDAD Y ROBUSTEZ:
   • Variabilidad F1: {best_rf['statistics']['f1_std']:.4f} ({'muy estable' if best_rf['statistics']['f1_std'] < 0.002 else 'estable' if best_rf['statistics']['f1_std'] < 0.005 else 'moderadamente variable'})
   • Consistencia entre particiones: {'Alta' if best_rf['statistics']['f1_std'] < 0.003 else 'Media'}
   • Confiabilidad para producción: {'Alta' if best_rf['statistics']['f1_std'] < 0.005 else 'Media'}

4. COMPARACIÓN ENTRE CONFIGURACIONES:
   • Diferencia de rendimiento: {abs(rf_f1_means[0] - rf_f1_means[1]):.4f}
   • {'Configuraciones muy similares' if abs(rf_f1_means[0] - rf_f1_means[1]) < 0.001 else 'Diferencia notable' if abs(rf_f1_means[0] - rf_f1_means[1]) > 0.005 else 'Diferencia moderada'}
   • Recomendación: {'Usar configuración más eficiente' if abs(rf_f1_means[0] - rf_f1_means[1]) < 0.001 else 'Usar configuración con mejor rendimiento'}

5. APLICABILIDAD EN DETECCIÓN DE INTRUSIONES:
   • Adecuado para IDS en tiempo real: {'Sí' if best_n < 150 and best_fpr < 0.01 else 'Con limitaciones' if best_n > 200 else 'Moderadamente'}
   • Tasa de falsas alarmas: {'Muy baja, apropiada para entornos críticos' if best_fpr < 0.005 else 'Baja, apropiada para uso general'}
   • Balance precision-recall: {'Óptimo para detección de ataques diversos' if best_f1 > 0.995 else 'Bueno para la mayoría de casos'}

6. JUSTIFICACIÓN DEL RANGO UTILIZADO:
   • El rango [10, 300] demostró ser apropiado
   • Valor óptimo {best_n} {'confirma límites conservadores' if best_n < 100 else 'valida rango medio' if best_n < 200 else 'justifica límite superior'}
   • {'No se observó plateau significativo' if best_n > 200 else 'Plateau alcanzado en rango medio' if best_n > 100 else 'Configuración eficiente encontrada'}

RECOMENDACIÓN FINAL:
Random Forest #{best_rf['architecture_rank']} con {best_n} árboles es la configuración
óptima, proporcionando F1-score de {best_f1:.4f} con baja tasa de falsos positivos
({best_fpr:.4f}), siendo {'altamente recomendado' if best_f1 > 0.995 and best_fpr < 0.005 else 'recomendado'}
para sistemas de detección de intrusiones en el dataset KDD99.
""")

In [ ]:
# Comparación entre mejor árbol de decisión y mejor Random Forest
print(f"\n{'=' * 80}")
print("Comparación: Mejor Árbol de Decisión vs Mejor Random Forest")
print("=" * 80)

# Verificar si existe la variable best_arch de la sección anterior de árboles de decisión
try:
    print("RENDIMIENTO:")
    print(f"  Árbol de Decisión  - F1: {best_arch['statistics']['f1_mean']:.4f} ± {best_arch['statistics']['f1_std']:.4f}")
    print(f"  Random Forest      - F1: {best_rf['statistics']['f1_mean']:.4f} ± {best_rf['statistics']['f1_std']:.4f}")
    print(f"  Mejora RF vs Árbol: {((best_rf['statistics']['f1_mean'] - best_arch['statistics']['f1_mean']) / best_arch['statistics']['f1_mean'] * 100):+.2f}%")
    
    print(f"\nTASA DE FALSOS POSITIVOS:")
    print(f"  Árbol de Decisión  - FPR: {best_arch['statistics']['fpr_mean']:.4f} ± {best_arch['statistics']['fpr_std']:.4f}")
    print(f"  Random Forest      - FPR: {best_rf['statistics']['fpr_mean']:.4f} ± {best_rf['statistics']['fpr_std']:.4f}")
    print(f"  Reducción FPR: {((best_arch['statistics']['fpr_mean'] - best_rf['statistics']['fpr_mean']) / best_arch['statistics']['fpr_mean'] * 100):+.2f}%")
    
    print(f"\nCOMPLEJIDAD COMPUTACIONAL:")
    print(f"  Árbol de Decisión: {best_arch['params']['max_depth']} niveles de profundidad")
    print(f"  Random Forest: {best_rf['params']['n_estimators']} árboles")
    
except NameError:
    print("No se puede comparar con árboles de decisión (variable best_arch no encontrada)")
    print("Ejecute primero la sección de árboles de decisión para hacer la comparación")

print(f"\n{'=' * 80}")
print("Comentarios sobre los resultados de Random Forest")
print("=" * 80)

print(f"""
ANÁLISIS DE RENDIMIENTO DE RANDOM FOREST:

1. EFECTIVIDAD DEL ENSEMBLE:
   - Los Random Forests muestran rendimiento {'superior' if best_rf['statistics']['f1_mean'] > 0.995 else 'excelente'} (F1 > {best_rf['statistics']['f1_mean']:.3f})
   - {'Reducción significativa' if best_rf['statistics']['fpr_mean'] < 0.005 else 'Control efectivo'} de falsos positivos
   - Estabilidad {'muy alta' if best_rf['statistics']['f1_std'] < 0.002 else 'alta'} entre diferentes particiones

2. HIPERPARÁMETROS ÓPTIMOS ENCONTRADOS:
   - n_estimators = {best_rf['params']['n_estimators']}: {'Suficiente para capturar patrones complejos' if best_rf['params']['n_estimators'] >= 100 else 'Equilibrio entre rendimiento y eficiencia'}

3. VENTAJAS OBSERVADAS:
   - Robustez ante overfitting por efecto ensemble
   - Manejo natural de características irrelevantes
   - Estimación de importancia de características
   - {'Paralelización eficiente' if best_rf['params'].get('n_jobs', 1) == -1 else 'Procesamiento secuencial'}

4. APLICABILIDAD EN DETECCIÓN DE INTRUSIONES:
   - Excelente para detección en tiempo real (baja latencia de predicción)
   - Tasa de falsas alarmas muy baja ({'<' if best_rf['statistics']['fpr_mean'] < 0.01 else '≈'}{best_rf['statistics']['fpr_mean']:.3f})
   - Alta precisión reduce ataques no detectados
   - Interpretabilidad mediante importancia de características

5. JUSTIFICACIÓN DE RANGO DE n_estimators:
   - Rango [10, 300] demostró ser apropiado
   - Valor óptimo {best_rf['params']['n_estimators']} {'en rango medio' if 50 <= best_rf['params']['n_estimators'] <= 200 else 'en extremo del rango'}
   - {'No se observó mejora significativa' if best_rf['params']['n_estimators'] < 200 else 'Se beneficia de mayor ensemble'} con más árboles
   
6. ESTABILIDAD Y CONFIABILIDAD:
   - Coeficiente de variación: {(best_rf['statistics']['f1_std'] / best_rf['statistics']['f1_mean']):.4f}
   - {'Muy estable' if (best_rf['statistics']['f1_std'] / best_rf['statistics']['f1_mean']) < 0.005 else 'Estable'} entre particiones diferentes
   - Intervalos de confianza estrechos indican alta confiabilidad

RECOMENDACIÓN:
Random Forest #{best_rf['architecture_rank']} es la configuración óptima encontrada, 
proporcionando el mejor balance entre rendimiento, estabilidad y eficiencia computacional
para la detección de intrusiones en el dataset KDD99.
""")